In [66]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

In [67]:
def linear_congruential_generator(N, seed = 1):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = seed

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples

## Test Script:
# N = 10
# print(linear_congruential_generator(N))

In [68]:
def inverse_normal_approximation(u):
    """Generates normally distributed realizations from uniform random realizations."""
    
    ## Constants for approximations to inverse normal.
    a0, a1, a2, a3 = 2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637
    b0, b1, b2, b3 = -8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833
    c0, c1, c2, c3 = 0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863
    c4, c5, c6, c7, c8 = 0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364, 0.0000003960315187

    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4 + r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x

    return x

## Test Script:
# u = 0.8
# print(inverse_normal_approximation(u))
## Results verified here: https://statisticshelper.com/inverse-normal-distribution-calculator/#answer

## Test Script 2:
# N = 1000
# k = 5
# z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
# plt.hist(z)

In [69]:
def box_muller(u1,u2):
    u1 = 2 * u1 - 1
    u2 = 2 * u2 - 1
    x = u1**2 + u2**2
    if x > 1:
        return None
    y = np.sqrt(-2*math.log(x)/x)
    z1 = u1 * y
    z2 = u2 * y
    return (z1,z2)

# Variance Reduction Techniques for Monte Carlo Pricing of European Options

In [70]:
def marsaglia_bray(seed=1):
    X = 2  # set an initial value so that the while loop is entered
    while X > 1:
        u = linear_congruential_generator(2, seed)
        u1, u2 = u[0], u[1]
        
        u1 = 2 * u1 - 1
        u2 = 2 * u2 - 1
        X = u1**2 + u2**2

        seed += 1
    
    Y = math.sqrt(-2 * math.log(X) / X)
    Z1 = u1 * Y
    Z2 = u2 * Y
    
    return Z1, Z2, seed

## Test Script:
# print(marsaglia_bray())

In [71]:
def marsaglia_bray_N(N):

    ## Generate an empty array for the generated values.
    results = np.empty(N, dtype=float)
    index = 0
    seed = 1

    while index < N:
        Z1, Z2, seed = marsaglia_bray(seed)
        
        ## Add the generated values to the results array.
        results[index] = Z1
        index += 1
        
        ## If there's still space, add the second generated value.
        if index < N:
            results[index] = Z2
            index += 1

    return results

## Test Script:
# N = 100_000
# plt.hist(marsaglia_bray_N(N), bins=20)

In [72]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

In [73]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

## Test Script:
# S = 100
# K = 100
# T = 1
# r = 0.03
# q = 0.03
# sigma = 0.2

# print("Call Option Value:", bs_call(S, K, T, r, q, sigma))
# print("Put Option Value:", bs_put(S, K, T, r, q, sigma))
# print("Call Option Delta:", bs_call_delta(S, K, T, r, q, sigma))
# print("Put Option Delta:", bs_put_delta(S, K, T, r, q, sigma))
# print("Option Gamma:", bs_gamma(S, K, T, r, q, sigma))
# print("Option Vega:", bs_vega(S, K, T, r, q, sigma))
## Verified results are correct here: https://goodcalculators.com/black-scholes-calculator/

In [74]:
def HW2_NUM1_CV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)
    ##plt.hist(Si_array, bins=10)
    
    ## Vectorized calculation for put payoff.
    Vi_array = put_payoff(Si_array, K, T, r)
    ##plt.hist(Vi_array, bins=10)

    ## Aggregation.
    S_hat = np.mean(Si_array)
    V_hat = np.mean(Vi_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_array - S_hat) * (Vi_array - V_hat)) / np.sum((Si_array - S_hat)**2)
    Wi_array = Vi_array - b_hat * (Si_array - np.exp(r * T) * S0)
    V_cv_hat = np.mean(Wi_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV_hat(N):", V_cv_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cv_hat))
    print("\n")

In [75]:
## Table values for control variate technique.

## Given values.
S0 = 56
K = 54
sigma = 0.27
q = 0
r = 0.02
T = 0.75
N = 10_000 * (2**9)

z = marsaglia_bray_N(N)

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV_hat(N): -0.08829437565900962
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 3.8893662594858456


Results for N = 20000
V_CV_hat(N): 1.11662504911072
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 2.684446834716116


Results for N = 40000
V_CV_hat(N): 3.2360875263414886
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.5649843574853475


Results for N = 80000
V_CV_hat(N): 4.003252952564317
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.20218106873748098


Results for N = 160000
V_CV_hat(N): 3.9601816612608234
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.15910977743398735


Results for N = 320000
V_CV_hat(N): 3.8950360541609865
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.0939641703341505


Results for N = 640000
V_CV_hat(N): 3.809412150533354
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.008340266706517774


Results for N = 1280000
V_CV_hat(N): 3.8126782297052864
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.01160634587845033


Results for N = 2560000
V

In [76]:
def HW2_NUM1_AV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si1_array = simulated_spot(S0, T, r, q, sigma, z)
    Si2_array = simulated_spot(S0, T, r, q, sigma, -z)
    
    ## Antithetic Variables Technique.
    Vi1_array = put_payoff(Si1_array, K, T, r)
    Vi2_array = put_payoff(Si2_array, K, T, r)
    V_av_hat = (np.mean(Vi1_array) + np.mean(Vi2_array)) / 2

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_AV_hat(N):", V_av_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_AV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_av_hat))
    print("\n")

In [77]:
## Table values for anthithetic variables technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_AV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_AV_hat(N): 1.986764384563329
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 1.814307499263507


Results for N = 20000
V_AV_hat(N): 2.7898757363983395
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 1.0111961474284965


Results for N = 40000
V_AV_hat(N): 3.751961837100339
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.0491100467264971


Results for N = 80000
V_AV_hat(N): 3.9635384125716886
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.16246652874485257


Results for N = 160000
V_AV_hat(N): 3.920028218358468
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.11895633453163201


Results for N = 320000
V_AV_hat(N): 3.8605760974415517
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.059504213614715695


Results for N = 640000
V_AV_hat(N): 3.7984789694442957
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.002592914382540279


Results for N = 1280000
V_AV_hat(N): 3.810458439678693
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.009386555851857104


Results for N = 2560000


In [78]:
def HW2_NUM1_MM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_MM_hat(N):", V_mm_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_mm_hat))
    print("\n")

In [79]:
## Table values for moment matching technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_MM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_MM_hat(N): 1.1933526453633463
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 2.6077192384634897


Results for N = 20000
V_MM_hat(N): 1.8240470097149772
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 1.9770248741118588


Results for N = 40000
V_MM_hat(N): 3.0179216460523652
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.7831502377744708


Results for N = 80000
V_MM_hat(N): 3.993431056009634
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.19235917218279797


Results for N = 160000
V_MM_hat(N): 3.946883828139249
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.14581194431241284


Results for N = 320000
V_MM_hat(N): 3.8781502112108006
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.07707832738396458


Results for N = 640000
V_MM_hat(N): 3.793159187329982
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.00791269649685411


Results for N = 1280000
V_MM_hat(N): 3.812279758797451
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.011207874970614817


Results for N = 2560000
V

In [80]:
def HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_tilda_array - np.exp(r*T)*S0) * (Vi_tilda_array - V_mm_hat)) / np.sum((Si_tilda_array - np.exp(r*T)*S0)**2)
    Wi_array = Vi_tilda_array - b_hat * (Si_tilda_array - np.exp(r * T) * S0)
    V_cvmm_hat = np.mean(Wi_array)
    
    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV,MM_hat(N):", V_mm_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV,MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cvmm_hat))
    print("\n")

In [81]:
## Table values for simultaneous moment matching and control variates technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV,MM_hat(N): 1.1933526453633463
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 2.6077192384634875


Results for N = 20000
V_CV,MM_hat(N): 1.8240470097149772
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 1.9770248741118595


Results for N = 40000
V_CV,MM_hat(N): 3.0179216460523652
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.7831502377744721


Results for N = 80000
V_CV,MM_hat(N): 3.993431056009634
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.19235917218279797


Results for N = 160000
V_CV,MM_hat(N): 3.946883828139249
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.14581194431241284


Results for N = 320000
V_CV,MM_hat(N): 3.8781502112108006
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.07707832738396236


Results for N = 640000
V_CV,MM_hat(N): 3.793159187329982
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.00791269649685633


Results for N = 1280000
V_CV,MM_hat(N): 3.812279758797451
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 

# Monte Carlo Pricing for Basket Options

# Monte Carlo Pricing for Path–Dependent Basket Options

In [82]:
def payoff_lockback_basket_call_option(S_1, S_2, K):
    return np.max(S_1+S_2-K, 0)

In [83]:
# parameter of the asset
S_1_0 = 26
S_2_0 = 29
sigma_1 = 0.31
sigma_2 = 0.21
rho = 0.3
r = 0.025
K = 50

T = 0.5


In [84]:
# page 6
df_result_page_6 = pd.DataFrame(columns = ['N_k' , 'm=150' , 'n' , 'V^(n)'])
m= 150 # fixed discretization
k = list(range(10))
delta_t = T / m

for _k in k:
    sum_V = 0
    n = 50*(2**_k) # paths
    N_k = 2 * m * n # 2 * 150 * 50 * 2**k # number of normal rvs required
    U = linear_congruential_generator(int(1.5 * N_k))
    count = 0
    i = 0
    while count < m * n:
        time = 0
        S_1 = S_1_0
        S_2 = S_2_0
        while time < m:
            u1 = U[2*i]
            u2 = U[2*i+1]
            i += 1
            tup = box_muller(u1, u2)
            if tup != None:
                zi, zj = tup
                count += 1
                S_1 = S_1 * np.exp((r-sigma_1**2/2)*delta_t + sigma_1*np.sqrt(delta_t)*zi)
                S_2 = S_2 * np.exp((r-sigma_2**2/2)*delta_t + sigma_2*np.sqrt(delta_t)*(rho*zi+np.sqrt(1-rho**2)*zj))
                time += 1
        
        sum_V += payoff_lockback_basket_call_option(S_1, S_2, K)
            
    avr_V = sum_V / n
        
    data = {
        'N_k': N_k,  
        'm=150': 150,  # fixed value
        'n': n, 
        '# of samples': count,
        'V^(n)': avr_V
    }
    df_result_page_6 = pd.concat([df_result_page_6, pd.DataFrame([data])], ignore_index=True)

In [85]:
df_result_page_6

,N_k,m=150,n,V^(n),# of samples
0,15000,150,50,4.983192,7500.000000
1,30000,150,100,6.357212,15000.000000
2,60000,150,200,6.020544,30000.000000
3,120000,150,400,5.387809,60000.000000
4,240000,150,800,5.532017,120000.000000
5,480000,150,1600,5.541333,240000.000000
6,960000,150,3200,5.710508,480000.000000
7,1920000,150,6400,5.593692,960000.000000
8,3840000,150,12800,5.662267,1920000.000000
9,7680000,150,25600,5.695043,3840000.000000


**Comment on the convergence of the Monte Carlo pricer:** 

A larger number of paths benefits the MC in converging, and the converge benefit margin decreases in incresing the number of paths.

# Monte Carlo Simulation for the Heston Model

In [86]:
#The parameters for the asset
S0 = 50
sigma = 0.3
V0 = 0.09
lam = 4
V_mean = 0.35**2
eta = 0.25
q = 0
rho = -0.15

#The parameters for the put
T = 0.5
K = 50
r = 0.05

#The parameters for the model
m = 175
delta_t = T / m


In [87]:
p_true = bs_put(S0, K, T, r, q, sigma)
print(p_true)

3.5829339156412274


In [88]:
values = []
for k in range(6):
    n = 500 * (2**k)
    N = 2 * 175 * n # number of independent normal rvs needed
    U = linear_congruential_generator(int(1.5*N)) # To generate N independent normal rvs, approximately needs 1.5*N independent uniform rvs
    sum_v = 0
    i = 0
    count = 0
    while count < 175 * n:
        time = 0
        S = S0
        V = V0
        while time < 175:
            u1 = U[2*i]
            u2 = U[2*i+1]
            i += 1
            tup = box_muller(u1,u2)
            if tup != None:
                zi,zj = tup
                count += 1
                S = S * np.exp((r - max(V,0)/2) * delta_t + np.sqrt(max(V,0) * delta_t) * zi)
                V = max(V,0) - lam * (max(V,0) - V_mean) * delta_t + eta * np.sqrt(max(V,0) * delta_t) * (rho * zi + np.sqrt(1 - rho**2) * zj)
                time += 1
        sum_v += put_payoff(S, K, T, r)
    value_hat = sum_v / n
    values.append(value_hat)

In [89]:
values

[3.9588521348855634,
 3.735216769947578,
 3.9614099515942423,
 4.0256382375400435,
 4.014398561331075,
 3.98149125780237]

In [90]:
#find the impied volatility
def bisection(f,x1,x2,tol):
    while max(abs(f(x1)),abs(f(x2)))>tol:
        mid = (x1+x2)/2
        if f(mid)*f(x1) < 0:
            x2 = mid
        else:
            x1 = mid
    return x1

In [91]:
imps = []
for v in values:
    def implied(sigma0):
        return bs_put(S0, K, T, r, q, sigma0) - v
    #print(implied(0.3))
    #print(implied(0.4))
    imp = bisection(implied,0.3,0.5,10**(-6))
    imps.append(imp)

In [92]:
imps

[0.3273270606994628,
 0.3110702037811279,
 0.3275129795074462,
 0.33218212127685537,
 0.3313650608062743,
 0.32897281646728516]